In [10]:
%load_ext autoreload
%autoreload 2

In [24]:
import matplotlib.pyplot as plt
from ptflops import get_model_complexity_info
from collections import defaultdict
import copy
import os

In [12]:
from CIFARnet import *

#### In this notebook we investigate the impact of dropout layers with small dropout rate on the perfomance of networks with close computational complexity but different depth/breadth ratios

In [80]:
def configure_ResNet(num_bundles=3, blocks_in_bundle=(3,3,3), channels=(3,16,16,32,64), dropout_rate_2d=0.1, weight_decay=0, batchnorm2d=(True,True)):
    """
        Configures hparams dictionary to be passed to the pytorch-lightning Module
    """
    assert num_bundles==len(blocks_in_bundle), print(f"'blocks_in_bundle' parameter should be of length 'num_bundles' ({num_bundles})")
    assert num_bundles==len(channels)-2, print(f"number of channels should be 'num_bundles'+2 {num_bundles+2}")
    assert num_bundles<6, print('Number of bundles can not be larger than 6')
    ### First convolutional Block
    FirstConvBlock = {'Cin':channels[0], 'Cout':channels[1], 'kernel_size':(3,3), 'stride':1, 'padding':1,
                 'dropout_rate':0,
                 'BatchNorm':True,
                 'activation':'relu'}

    ### Configure Residual Blocks
    ResBlockTemplate = {'kernel_size':(3,3),'stride':1, 'padding':1,
                     'dropout_rate':dropout_rate_2d, 'batchnorm2d':batchnorm2d, 'activation':'relu'}
    
    BundleTemplate = {'blocks_num':blocks_in_bundle, 'Cin_first':3, 'stride_first':2, 'block_hparams':ResBlockTemplate}
    Conv2dBundles = defaultdict(lambda: copy.deepcopy(BundleTemplate))
    
    for i in range(num_bundles):
        stride_first = 1 if i==0 else 2
        Conv2dBundles['cv_bundle'+str(i+1)].update({'blocks_num':blocks_in_bundle[i], 'Cin_first':channels[i+1],
                                                    'stride_first':stride_first})
        Conv2dBundles['cv_bundle'+str(i+1)]['block_hparams'].update({'Cin':channels[i+2], 'Cout':channels[i+2]})
    
    ### Configure global hyperparameters
    hparams = {'Conv2dBundles':dict(Conv2dBundles), 
               'FirstConvBlock':FirstConvBlock, 'FirstMaxPool':False,
               'FcBlock': (channels[-1], 10), 'out_of_conv_dim':(32//(2**(num_bundles-1)),32//(2**(num_bundles-1)))}
    hparams['batch_size'] = 100
    hparams['learning_rate'] = 2e-4
    hparams['weight_decay'] = weight_decay
    return hparams

In [81]:
from pytorch_lightning.callbacks import LearningRateLogger, EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
path = '/home/moonstrider/Neural_Networks_Experiments/VISION/CIFAR10/'
early_stopping = EarlyStopping('val_loss', min_delta=0.0, patience=3)
tmpdir = '/home/moonstrider/Neural_Networks_Experiments/VISION/CIFAR10/chkpt'

In [92]:
def set_trainer_and_logger(run_name, tags=[], project='Depth-Breadth-Dropout', offline=True, tmpdir=tmpdir):
    wandb_logger = WandbLogger(name=run_name, save_dir=path+'WandB_Logs-extra/',
                               offline=offline, project=project, tags=tags)
    model_chekpoint = ModelCheckpoint(os.path.join(tmpdir, f'{run_name}'+'-{epoch}-{val_loss:.2f}'), 
                                  verbose=True, period=3, save_top_k=1)
    trainer = pl.Trainer(gpus=1, max_epochs=22, logger=wandb_logger, callbacks=[early_stopping],
                         checkpoint_callback=None)
    return trainer, model_chekpoint

### Finding optimal parameters to have consistent FLOPS/Params-number steps (yes, by hand)

In [ ]:
macs_list, pars_list = [], []
for block_size in (3,9,15,21,27,33):
    config = configure_ResNet(num_bundles=3, blocks_in_bundle=(3,block_size,3), channels=(3,16,16,32,64))
    M = ResNetCifar(config)
    with torch.cuda.device(0):
        macs, pars = get_model_complexity_info(M, (3,32,32), as_strings=False, print_per_layer_stat=False, verbose=False)
        print(block_size, macs, pars)
        macs_list.append(macs)
        pars_list.append(pars)
print([x2-x1 for x1,x2 in zip(pars_list[:-1], pars_list[1:])])
print([x2-x1 for x1,x2 in zip(macs_list[:-1], macs_list[1:])])

macs_list, pars_list = [], []
for channel_mid in (32, 56, 74, 88, 100, 110):
    config = configure_ResNet(num_bundles=3, blocks_in_bundle=(3,3,3), channels=(3,16,16,channel_mid,64))
    M = ResNetCifar(config)
    with torch.cuda.device(0):
        macs, pars = get_model_complexity_info(M, (3,32,32), as_strings=False, print_per_layer_stat=False, verbose=False)
        print(block_size, macs, pars)
        macs_list.append(macs)
        pars_list.append(pars)
print([x2-x1 for x1,x2 in zip(pars_list[:-1], pars_list[1:])])
print([x2-x1 for x1,x2 in zip(macs_list[:-1], macs_list[1:])])

### Preparing model configs

In [93]:
Model_configs = {}
tags = []
for block_size in (3,9,15,21,27,33):
    for drop in (0, 0.02):
        config = configure_ResNet(num_bundles=3, blocks_in_bundle=(3,block_size,3),
                                  channels=(3,16,16,32,64), dropout_rate_2d=drop)
        Model_configs[f'ResNet-depth-mid-blocks-{block_size}-drop-{drop}'] = config
        tags.append(('depth', 'drop-Yes' if drop>0 else 'drop-No'))
for mid_ch in (32, 56, 74, 88, 100, 110):
    for drop in (0, 0.02):
        config = configure_ResNet(num_bundles=3, blocks_in_bundle=(3,3,3),
                                  channels=(3,16,16,mid_ch,64), dropout_rate_2d=drop)
        Model_configs[f'ResNet-breadth-mid-chan-{mid_ch}-drop-{drop}'] = config
        tags.append(('breadth', 'drop-Yes' if drop>0 else 'drop-No'))

### Running all models

In [110]:
i = 0
for run_name, tag in zip(Model_configs, tags):
    print('run_name:', run_name)
    print(f'run {i} out of {len(Model_configs)}, {i/len(Model_configs):.2f} finished!')
    trainer, model_chekpoint = set_trainer_and_logger(run_name, tags=tag)
    M = ResNetCifar(Model_configs[run_name])
    print('Starting training for 22 epochs with early stopping...\n')
    trainer.fit(M)
    i += 1
print('All runs finished!')

run_name: ResNet-breadth-mid-chan-56-drop-0
run 0 out of 8, 0.00 finished!


GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]


Starting training for 22 epochs with early stopping...

Files already downloaded and verified
Files already downloaded and verified


wandb: Wandb version 0.9.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade

   | Name                  | Type        | Params
--------------------------------------------------
0  | FirstConv             | ConvBlock   | 480   
1  | FirstConv.Conv2d      | Conv2d      | 448   
2  | FirstConv.BatchNorm2d | BatchNorm2d | 32    
3  | cv_bundle1            | Sequential  | 14 K  
4  | cv_bundle1.0          | ResNetBlock | 4 K   
5  | cv_bundle1.0.conv1    | Conv2d      | 2 K   
6  | cv_bundle1.0.conv2    | Conv2d      | 2 K   
7  | cv_bundle1.0.bn1      | BatchNorm2d | 32    
8  | cv_bundle1.0.bn2      | BatchNorm2d | 32    
9  | cv_bundle1.1          | ResNetBlock | 4 K   
10 | cv_bundle1.1.conv1    | Conv2d      | 2 K   
11 | cv_bundle1.1.conv2    | Conv2d      | 2 K   
12 | cv_bundle1.1.bn1      | BatchNorm2d | 32    
13 | cv_bundle1.1.bn2      | BatchNorm2d | 32    
14 | cv_bundle1.2          | ResNetBlock | 4 K   
15 | cv_bundle1.2.conv1    | Conv2d      | 2

wandb: WARNING Adding to old History rows isn't currently supported.  Step 0 < 1; dropping {'train_loss': 3.945166826248169, 'train_acc': 0.09, 'epoch': 0}.


...

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]



run_name: ResNet-breadth-mid-chan-100-drop-0.02
run 7 out of 8, 0.88 finished!
Starting training for 22 epochs with early stopping...

Files already downloaded and verified
Files already downloaded and verified


wandb: Wandb version 0.9.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade

   | Name                  | Type        | Params
--------------------------------------------------
0  | FirstConv             | ConvBlock   | 480   
1  | FirstConv.Conv2d      | Conv2d      | 448   
2  | FirstConv.BatchNorm2d | BatchNorm2d | 32    
3  | cv_bundle1            | Sequential  | 14 K  
4  | cv_bundle1.0          | ResNetBlock | 4 K   
5  | cv_bundle1.0.conv1    | Conv2d      | 2 K   
6  | cv_bundle1.0.conv2    | Conv2d      | 2 K   
7  | cv_bundle1.0.bn1      | BatchNorm2d | 32    
8  | cv_bundle1.0.bn2      | BatchNorm2d | 32    
9  | cv_bundle1.0.dropout1 | Dropout2d   | 0     
10 | cv_bundle1.0.dropout2 | Dropout2d   | 0     
11 | cv_bundle1.1          | ResNetBlock | 4 K   
12 | cv_bundle1.1.conv1    | Conv2d      | 2 K   
13 | cv_bundle1.1.conv2    | Conv2d      | 2 K   
14 | cv_bundle1.1.bn1      | BatchNorm2d | 32    
15 | cv_bundle1.1.bn2      | BatchNorm2d | 3

wandb: WARNING Adding to old History rows isn't currently supported.  Step 0 < 1; dropping {'train_loss': 2.946197748184204, 'train_acc': 0.15, 'epoch': 0}.



All runs finished!
